In [1]:
!pip install tf-agents tensorflow

In [2]:
#Initialization

In [3]:
import numpy as np
import tensorflow as tf
import requests
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.utils import common

BASE_URL = "http://127.0.0.1:5000"
REPLAY_BUFFER_MAX_LENGTH = 100000  
BATCH_SIZE = 64  
LEARNING_RATE = 1e-3  
NUM_ITERATIONS = 20000  
COLLECT_STEPS_PER_ITERATION = 1  
LOG_INTERVAL = 200  
EVAL_INTERVAL = 1000  



2023-09-16 10:30:47.976434: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 10:30:48.014726: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 10:30:48.015287: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 10:30:48.714912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Initialising the Environment and Fetching the Action and Observation Spec

In [4]:
from tf_agents.specs import array_spec


response = requests.post(f"{BASE_URL}/create_env", json={"race": "Shanghai_2019"})
data = response.json()
session_id = data['session_id']

print(session_id)


context = {
    "BoundedArraySpec": array_spec.BoundedArraySpec,
    "dtype": np.dtype  # Add this line
    # Add other specs if needed
}

import requests

response_action = requests.get(f"http://localhost:5000/action_spec?session_id={session_id}")
response_observation = requests.get(f"http://localhost:5000/observation_spec?session_id={session_id}")



action_spec = eval(response_action.json(), context)
observation_spec = eval(response_observation.json(), context)

print(action_spec)
print(observation_spec)


63b811ff-735c-4ba4-aead-06a8700fbc6b
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3)
BoundedArraySpec(shape=(40,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


# Initialising the Agent

In [5]:
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

dummy_time_step_spec = ts.time_step_spec(observation_spec)


q_net = q_network.QNetwork(
    observation_spec,
    action_spec,
    fc_layer_params=(100,))

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=LEARNING_RATE)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    time_step_spec=dummy_time_step_spec,
    action_spec=action_spec,
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)
agent.initialize()
print("Agent initialized!")

Agent initialized!


In [6]:
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

# Preserving the time step spec from the given code
dummy_time_step_spec = ts.time_step_spec(observation_spec)

learning_rate= 1e-4
num_iterations = 250_000
n_step_update = 1
fc_layer_params=(64,64,)
target_update_period = 1
gamma = 1.0

# Using the QNetwork definition from your provided code, and added fc_layer_params
q_net = q_network.QNetwork(
    input_tensor_spec=observation_spec,
    action_spec=action_spec,
    fc_layer_params=fc_layer_params)

# Changed optimizer to tf.keras.optimizers.Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_step_counter = tf.Variable(0)

"""

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# Polynomial Decay for Boltzmann exploration (as in the previous code)
boltzmann_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0,
    decay_steps=num_iterations,
    end_learning_rate=0.05)
    
## Getting good for the below

# Polynomial Decay for Boltzmann exploration (as in the previous code)
boltzmann_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.3,
    decay_steps=num_iterations,
    end_learning_rate=0.05)
"""
# Polynomial Decay for Boltzmann exploration (as in the previous code)
boltzmann_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.4,
    decay_steps=num_iterations,
    end_learning_rate=0.01)

# Modifying the agent initialization to match the provided code
agent_boltz_policy = dqn_agent.DqnAgent(
    time_step_spec=dummy_time_step_spec,
    action_spec=action_spec,
    q_network=q_net,
    optimizer=optimizer,
    n_step_update=n_step_update,
    target_update_period=target_update_period,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter,
    epsilon_greedy=None,
    boltzmann_temperature=lambda: boltzmann_fn(train_step_counter))

agent_boltz_policy.initialize()
print("agent_boltz_policy  initialized!")


agent_boltz_policy  initialized!


In [7]:
print("Agent_boltz_policy Network:")
print(agent_boltz_policy._q_network.summary())



print("\nAction Spec:")
print(action_spec)

print("\nObservation Spec:")
print(observation_spec)


Agent_boltz_policy Network:
Model: "QNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncodingNetwork (EncodingN  multiple                  6784      
 etwork)                                                         
                                                                 
 dense_6 (Dense)             multiple                  260       
                                                                 
Total params: 7044 (27.52 KB)
Trainable params: 7044 (27.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3)

Observation Spec:
BoundedArraySpec(shape=(40,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


In [8]:
print("Agent Network:")
print(agent._q_network.summary())

print("\nAgent Learning Rate:")
print(agent._optimizer._lr)

print("\nAction Spec:")
print(action_spec)

print("\nObservation Spec:")
print(observation_spec)


Agent Network:
Model: "QNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncodingNetwork (EncodingN  multiple                  4100      
 etwork)                                                         
                                                                 
 dense_1 (Dense)             multiple                  404       
                                                                 
Total params: 4504 (17.59 KB)
Trainable params: 4504 (17.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None

Agent Learning Rate:
0.001

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3)

Observation Spec:
BoundedArraySpec(shape=(40,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


# Replay Buffer Initialisation

In [9]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

# Initialize the replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent_boltz_policy.collect_data_spec,
    batch_size=1,  #changed this from 1 to 64
    max_length=REPLAY_BUFFER_MAX_LENGTH)
"""
def collect_step_from_api(session_id, policy, buffer):
    # Get current state from API
    response = requests.get(f"{BASE_URL}/{session_id}/current_time_step")
    if response.status_code != 200:
        print("Error getting current time step:", response.content)
        return
    time_step_data = response.json()
    current_time_step = ts.restart(np.array(time_step_data["observation"], dtype=np.float32))
    
    # Get action from policy
    action_step = policy.action(current_time_step)
    
    # Send action to API
    response = requests.post(f"{BASE_URL}/{session_id}/step", json={"action": int(action_step.action.numpy())})
    if response.status_code != 200:
        print("Error sending action to API:", response.content)
        return
    next_time_step_data = response.json()
    next_time_step = ts.transition(
        np.array(next_time_step_data["observation"], dtype=np.float32),
        reward=next_time_step_data["reward"],
        discount=next_time_step_data["discount"])
    
    # Create trajectory and add to buffer
    traj = trajectory.from_transition(current_time_step, action_step, next_time_step)
    buffer.add_batch(traj)
"""

'\ndef collect_step_from_api(session_id, policy, buffer):\n    # Get current state from API\n    response = requests.get(f"{BASE_URL}/{session_id}/current_time_step")\n    if response.status_code != 200:\n        print("Error getting current time step:", response.content)\n        return\n    time_step_data = response.json()\n    current_time_step = ts.restart(np.array(time_step_data["observation"], dtype=np.float32))\n    \n    # Get action from policy\n    action_step = policy.action(current_time_step)\n    \n    # Send action to API\n    response = requests.post(f"{BASE_URL}/{session_id}/step", json={"action": int(action_step.action.numpy())})\n    if response.status_code != 200:\n        print("Error sending action to API:", response.content)\n        return\n    next_time_step_data = response.json()\n    next_time_step = ts.transition(\n        np.array(next_time_step_data["observation"], dtype=np.float32),\n        reward=next_time_step_data["reward"],\n        discount=next_time

# Data Collection with a Random Policy

In [10]:
from tf_agents.policies import random_tf_policy
from tf_agents.trajectories import trajectory, time_step as ts
import requests
import numpy as np

# Initialize a random policy
random_policy = random_tf_policy.RandomTFPolicy(
    agent_boltz_policy.time_step_spec, 
    agent_boltz_policy.action_spec
)

def create_time_step(step_type, reward, discount, observation):
    """Constructs a time_step with batched tensors."""
    return ts.TimeStep(
        step_type=tf.constant([step_type], dtype=tf.int32),
        reward=tf.constant([reward], dtype=tf.float32),
        discount=tf.constant([discount], dtype=tf.float32),
        observation=tf.constant(np.expand_dims(observation, axis=0), dtype=tf.float32)
    )


def collect_step_from_api(session_id, policy, buffer, state):
    time_step = create_time_step(ts.StepType.MID, 0.0, 1.0, state)  # Initial step
    #print(time_step)
    #return
    #print("+++++++======++++++++++========>>>>>>>===")
    # Use the policy to select an action
    action = policy.action(time_step)
    #print(action)
    
    # Convert action tensor to numpy and then to native Python type
    action_value = int(action.action.numpy())
    
    # Take a step in the environment with the selected action
    response = requests.post(f"http://localhost:5000/{session_id}/step", json={"action": action_value})
    result = response.json()
    next_state = result["observation"]
    reward = result["reward"]
    done = result["done"]
    
    next_step_type = ts.StepType.LAST if done else ts.StepType.MID
    next_time_step = create_time_step(next_step_type, reward, 1.0, next_state)
    
    # Add experience to the replay buffer
    traj = trajectory.from_transition(time_step, action, next_time_step)

    # Ensure the trajectory has an extra batch dimension
    #traj = tf.nest.map_structure(lambda x: tf.expand_dims(x, 0), traj)
    
    buffer.add_batch(traj)
    
    return next_state




def collect_data_from_api(session_id, policy, buffer, steps):
    # Assuming you start from an initial state
    response = requests.post(f"http://localhost:5000/{session_id}/reset")
    state = response.json()["observation"]
    
    for _ in range(steps):
        state = collect_step_from_api(session_id, policy, buffer, state)

# Collect initial data with random policy
initial_collect_steps = 1000
collect_data_from_api(session_id, random_policy, replay_buffer, initial_collect_steps)


In [11]:
print(replay_buffer.num_frames())

tf.Tensor(1000, shape=(), dtype=int64)


In [12]:
sample_data = replay_buffer.gather_all()
print(sample_data)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.
Trajectory(
{'action': <tf.Tensor: shape=(1, 1000), dtype=int32, numpy=
array([[1, 3, 1, 0, 0, 3, 0, 2, 1, 2, 0, 0, 0, 1, 3, 1, 3, 1, 1, 1, 2, 1,
        1, 1, 2, 0, 0, 2, 1, 2, 2, 0, 1, 2, 3, 0, 2, 2, 1, 1, 0, 3, 3, 2,
        2, 0, 0, 1, 0, 1, 0, 0, 2, 3, 3, 2, 1, 1, 1, 1, 3, 2, 3, 3, 3, 2,
        1, 1, 1, 2, 3, 1, 1, 2, 0, 2, 0, 3, 0, 0, 1, 0, 3, 1, 0, 1, 1, 2,
        2, 1, 3, 0, 3, 3, 0, 3, 1, 2, 3, 2, 2, 0, 0, 2, 3, 3, 1, 0, 3, 2,
        0, 2, 0, 2, 2, 2, 0, 2, 0, 3, 1, 1, 0, 0, 1, 3, 3, 3, 1, 2, 3, 0,
        3, 0, 2, 0, 1, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0, 1, 0, 1, 2, 1, 3,
        1, 3, 1, 1, 0, 0, 3, 1, 3, 0, 3, 2, 1, 1, 0, 1, 3, 1, 1, 3, 1, 2,
        2, 1, 2, 1, 3, 2, 1, 3, 0, 2, 1, 3, 0, 1, 1, 0, 2, 3, 0, 3, 2, 0,
        0, 1, 1, 3, 1, 3, 1, 1, 2, 1, 0, 1, 2, 2, 1, 0, 0, 2, 1, 0, 2, 3,
        0, 3, 3, 3, 0, 0, 3, 0, 0, 2, 0, 1, 0, 3, 2, 0, 2, 2, 0, 2, 2, 0,
        3, 2, 0, 1, 3, 3

# DQN Attempt 1

In [13]:
#Initialise DQN Policy
dqn_policy = agent_boltz_policy.policy

print(agent_boltz_policy.policy)

#Initialise Replay Buffer for DQN
#from tf_agents.replay_buffers import tf_uniform_replay_buffer

# Initialize the replay buffer
#replay_buffer_dqn = tf_uniform_replay_buffer.TFUniformReplayBuffer(
#    data_spec=agent_boltz_policy.collect_data_spec,
#    batch_size=1,  # Single environment: the API
#    max_length=REPLAY_BUFFER_MAX_LENGTH)

In [14]:
dataset = replay_buffer.as_dataset(num_parallel_calls=3,
                                   sample_batch_size=1,
                                   num_steps=2).prefetch(3)  # Assuming n_step_update = 1 for this example

dataset_iterator = iter(dataset)

sample_observation = next(dataset_iterator)[0].observation.numpy()
observation_dim = sample_observation.shape
print(f"Observation Dimension: {observation_dim}")






Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Observation Dimension: (1, 2, 40)


In [15]:
collect_data_from_api(session_id, dqn_policy, replay_buffer, initial_collect_steps)

In [16]:
def printme():
    print("name")

In [17]:
printme()

name


In [18]:
"""
batch_size = 60
trajectory_length = 2
# Function to reshape experience
def reshape_experience_for_dqn(experience):
    # Determine the actual batch size of the experience
    actual_batch_size = tf.shape(experience.step_type)[0]
    
    # Determine the adjusted sample size based on the actual batch size
    adjusted_sample_size = actual_batch_size // trajectory_length
    
    # Reshape the experience to [adjusted_sample_size, trajectory_length] + spec.shape to form trajectories
    reshaped_experience = tf.nest.map_structure(
        lambda x: tf.reshape(x, [adjusted_sample_size, trajectory_length] + list(x.shape[1:])),
        experience
    )
    return reshaped_experience

# Number of iterations to train the agent
num_iterations = 20000

# Interval for logging to the console
log_interval = 1000

# Metrics to track during training
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)

for iteration in range(num_iterations):
    
    # Sample a batch of experience from the replay buffer
    experience, _ = replay_buffer.get_next(sample_batch_size=batch_size)
    experience = reshape_experience_for_dqn(experience)
    
    # Train the agent using the sampled experience
    train_loss_value = agent.train(experience).loss
    
    # Update training loss metric
    train_loss(train_loss_value)
    
    # Logging and evaluation
    if iteration % log_interval == 0:
        print(f"Iteration {iteration}")
        print(f"Training loss: {train_loss.result()}")
        train_loss.reset_states()
"""

'\nbatch_size = 60\ntrajectory_length = 2\n# Function to reshape experience\ndef reshape_experience_for_dqn(experience):\n    # Determine the actual batch size of the experience\n    actual_batch_size = tf.shape(experience.step_type)[0]\n    \n    # Determine the adjusted sample size based on the actual batch size\n    adjusted_sample_size = actual_batch_size // trajectory_length\n    \n    # Reshape the experience to [adjusted_sample_size, trajectory_length] + spec.shape to form trajectories\n    reshaped_experience = tf.nest.map_structure(\n        lambda x: tf.reshape(x, [adjusted_sample_size, trajectory_length] + list(x.shape[1:])),\n        experience\n    )\n    return reshaped_experience\n\n# Number of iterations to train the agent\nnum_iterations = 20000\n\n# Interval for logging to the console\nlog_interval = 1000\n\n# Metrics to track during training\ntrain_loss = tf.keras.metrics.Mean(\'train_loss\', dtype=tf.float32)\n\nfor iteration in range(num_iterations):\n    \n    # S

In [19]:
batch_size = 60
trajectory_length = 2
# Function to reshape experience
def reshape_experience_for_dqn(experience):
    # Determine the actual batch size of the experience
    actual_batch_size = tf.shape(experience.step_type)[0]
    
    # Determine the adjusted sample size based on the actual batch size
    adjusted_sample_size = actual_batch_size // trajectory_length
    
    # Reshape the experience to [adjusted_sample_size, trajectory_length] + spec.shape to form trajectories
    reshaped_experience = tf.nest.map_structure(
        lambda x: tf.reshape(x, [adjusted_sample_size, trajectory_length] + list(x.shape[1:])),
        experience
    )
    return reshaped_experience

# Number of iterations to train the agent
num_iterations = 20000

# Interval for logging to the console
log_interval = 1000


# (optional) optimize by wrapping some of the code in a graph using TF function
agent_boltz_policy.train = common.function(agent_boltz_policy.train)


# Metrics to track during training
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)


for iteration in range(num_iterations):
    
    # Collect experiences
    #for _ in range(collect_steps_per_iteration):
    collect_data_from_api(session_id, agent_boltz_policy.collect_policy, replay_buffer, 10)
    
    # Sample a batch of experience from the replay buffer
    experience, _ = replay_buffer.get_next(sample_batch_size=batch_size)
    experience = reshape_experience_for_dqn(experience)
    
    # Train the agent using the sampled experience
    train_loss_value = agent_boltz_policy.train(experience).loss
    
    # Update training loss metric
    train_loss(train_loss_value)
    
    # Logging and evaluation
    if iteration % log_interval == 0:
        print(f"Iteration {iteration}")
        print(f"Training loss: {train_loss.result()}")
        train_loss.reset_states()


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
Iteration 0
Training loss: 3.8004112243652344
Iteration 1000
Training loss: 2.718580722808838
Iteration 2000
Training loss: 0.606881320476532
Iteration 3000
Training loss: 0.4349765479564667
Iteration 4000
Training loss: 0.5111838579177856
Iteration 5000
Training loss: 1.7186973094940186
Iteration 6000
Training loss: 3.112914800643921
Iteration 7000
Training loss: 4.202648162841797
Iteration 8000
Training loss: 6.114943981170654
Iteration 9000
Training loss: 6.384117603302002
Iteration 10000
Training loss: 7.515146255493164
Iteration 11000
Training loss: 7.629866600036621
Iteration 12000
Training loss: 6.353353023529053
Iteration 13000
Training loss: 5.666082382202148
Iteration 14000
Training loss: 5.708736896514893
Iteration 15000
Training loss: 5.8421874

In [20]:
def compute_average_return(session_id, policy, num_episodes=1):
    total_return = 0.0

    for _ in range(num_episodes):
        # Reset the environment to start a new episode
        response = requests.post(f"{BASE_URL}/{session_id}/reset")
        if response.status_code != 200:
            print("Error resetting the environment:", response.content)
            return None
        state = response.json()["observation"]
        episode_return = 0.0
        done = False

        # Interact with the environment using the policy until the episode terminates
        while not done:
            # Convert the state to a time step
            current_time_step = ts.restart(np.array([state], dtype=np.float32))
            
            # Use the policy to select an action
            action_step = policy.action(current_time_step)
            action_value = int(action_step.action.numpy())
            
            # Execute the action in the environment
            response = requests.post(f"{BASE_URL}/{session_id}/step", json={"action": action_value})
            if response.status_code != 200:
                print("Error taking an action in the environment:", response.content)
                return None
            result = response.json()
            state = result["observation"]
            reward = result["reward"]
            done = result["done"]
            
            # Accumulate the reward
            episode_return += reward

        total_return += episode_return

    # Compute the average return
    average_return = total_return / num_episodes

    return average_return


In [21]:
chosen_policy = agent_boltz_policy.policy

In [22]:
average_return = compute_average_return(session_id, chosen_policy, num_episodes=100)
print(f"Average Return over 100 episodes: {average_return}")

Average Return over 100 episodes: -18.84533268033528


# Dueling Networks TF Agent Implementation

## Agent with DDQN and Dueling Network Implementation

In [23]:
from tf_agents.networks import q_network
import tensorflow as tf

class DuelingQNetwork(q_network.QNetwork):
    def __init__(self, input_tensor_spec, action_spec, fc_layer_params, **kwargs):
        super(DuelingQNetwork, self).__init__(input_tensor_spec, action_spec, fc_layer_params=fc_layer_params, **kwargs)
        self._action_spec = action_spec  # Store the action spec

    def call(self, observation, step_type=None, network_state=(), training=False):
        # Pass the observation through the initial layers
        state = super(DuelingQNetwork, self).call(observation, step_type, network_state, training)
        
        # Dueling branches
        value = tf.keras.layers.Dense(1, activation=None)(state[0])
        advantages = tf.keras.layers.Dense(self._action_spec.maximum + 1, activation=None)(state[0])
        
        # Combine value and advantages into Q-values
        q_values = value + (advantages - tf.reduce_mean(advantages, axis=1, keepdims=True))
        
        return q_values, network_state




In [24]:
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

# Preserving the time step spec from the given code
dummy_time_step_spec = ts.time_step_spec(observation_spec)

learning_rate= 1e-3
num_iterations = 250_000
n_step_update = 1
fc_layer_params=(64,64,)
target_update_period = 1
gamma = 1.0

# Using the QNetwork definition from your provided code, and added fc_layer_params
dueling_q_net = DuelingQNetwork(
    input_tensor_spec=observation_spec,
    action_spec=action_spec,
    fc_layer_params=fc_layer_params)

# Changed optimizer to tf.keras.optimizers.Adam from the provided code
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_step_counter = tf.Variable(0)

# Polynomial Decay for Boltzmann exploration (as in the previous code)
boltzmann_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0,
    decay_steps=num_iterations,
    end_learning_rate=0.01)

# Modifying the agent initialization to match the provided code
agent_dueling_qnet = dqn_agent.DqnAgent(
    time_step_spec=dummy_time_step_spec,
    action_spec=action_spec,
    q_network=dueling_q_net,
    optimizer=optimizer,
    n_step_update=n_step_update,
    target_update_period=target_update_period,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter,
    epsilon_greedy=None,
    boltzmann_temperature=lambda: boltzmann_fn(train_step_counter))

agent_dueling_qnet.initialize()
print("agent_dueling_qnet  initialized!")


agent_dueling_qnet  initialized!


In [25]:
print("agent_dueling_qnet Network:")
print(agent_dueling_qnet._q_network.summary())



print("\nAction Spec:")
print(action_spec)

print("\nObservation Spec:")
print(observation_spec)

agent_dueling_qnet Network:
Model: "QNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncodingNetwork (EncodingN  multiple                  6784      
 etwork)                                                         
                                                                 
 dense_12 (Dense)            multiple                  260       
                                                                 
Total params: 7044 (27.52 KB)
Trainable params: 7044 (27.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3)

Observation Spec:
BoundedArraySpec(shape=(40,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


### Replay Buffer for Dueling Q Net Implementation

In [26]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

# Initialize the replay buffer
replay_buffer_duel = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent_dueling_qnet.collect_data_spec,
    batch_size=1,  
    max_length=REPLAY_BUFFER_MAX_LENGTH)

### Data Collection with Random Policy to Populate Dueling Q Net Implementation's Replay Buffer

In [27]:
from tf_agents.policies import random_tf_policy
from tf_agents.trajectories import trajectory, time_step as ts
import requests
import numpy as np

# Initialize a random policy
random_policy = random_tf_policy.RandomTFPolicy(
    agent_dueling_qnet.time_step_spec, 
    agent_dueling_qnet.action_spec
)

def create_time_step(step_type, reward, discount, observation):
    """Constructs a time_step with batched tensors."""
    return ts.TimeStep(
        step_type=tf.constant([step_type], dtype=tf.int32),
        reward=tf.constant([reward], dtype=tf.float32),
        discount=tf.constant([discount], dtype=tf.float32),
        observation=tf.constant(np.expand_dims(observation, axis=0), dtype=tf.float32)
    )


def collect_step_from_api(session_id, policy, buffer, state):
    time_step = create_time_step(ts.StepType.MID, 0.0, 1.0, state)  # Initial step
    print(time_step)
    #return
    #print("+++++++======++++++++++========>>>>>>>===")
    # Use the policy to select an action
    action = policy.action(time_step)
    print(action)
    
    # Convert action tensor to numpy and then to native Python type
    action_value = int(action.action.numpy())
    
    # Take a step in the environment with the selected action
    response = requests.post(f"http://localhost:5000/{session_id}/step", json={"action": action_value})
    result = response.json()
    next_state = result["observation"]
    reward = result["reward"]
    done = result["done"]
    
    next_step_type = ts.StepType.LAST if done else ts.StepType.MID
    next_time_step = create_time_step(next_step_type, reward, 1.0, next_state)
    
    # Add experience to the replay buffer
    traj = trajectory.from_transition(time_step, action, next_time_step)

    # Ensure the trajectory has an extra batch dimension
    #traj = tf.nest.map_structure(lambda x: tf.expand_dims(x, 0), traj)
    
    buffer.add_batch(traj)
    
    return next_state




def collect_data_from_api(session_id, policy, buffer, steps):
    # Assuming you start from an initial state
    response = requests.post(f"http://localhost:5000/{session_id}/reset")
    state = response.json()["observation"]
    
    for _ in range(steps):
        state = collect_step_from_api(session_id, policy, buffer, state)

# Collect initial data with random policy
initial_collect_steps = 1000
collect_data_from_api(session_id, random_policy, replay_buffer_duel, initial_collect_steps)


TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.01785714, 0.2631579 , 0.7368421 , 0.05357143, 0.        ,
        0.        , 1.        , 1.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.19642857, 1.        , 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.35714287, 1.        , 0.        , 0.03571429, 0.        ,
        1.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.5714286 , 1.        , 0.        , 0.03571429, 1.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.75      , 0.94736844, 0.        , 0.01785714, 1.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

...

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.78571427, 0.94736844, 0.        , 0.05357143, 0.        ,
        1.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.03571429, 0.94736844, 0.        , 0.00446429, 1.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        1.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.26785713, 0.94736844, 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.53571427, 0.94736844, 0.        , 0.01785714, 0.        ,
        1.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.8035714 , 0.94736844, 0.        , 0.01785714, 1.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), inf

In [28]:
print(replay_buffer_duel.num_frames())

tf.Tensor(1000, shape=(), dtype=int64)


In [29]:
sample_data = replay_buffer_duel.gather_all()
print(sample_data)

Trajectory(
{'action': <tf.Tensor: shape=(1, 1000), dtype=int32, numpy=
array([[2, 1, 0, 2, 1, 2, 0, 1, 3, 3, 1, 2, 2, 3, 3, 0, 0, 2, 0, 1, 1, 2,
        1, 0, 3, 1, 3, 0, 1, 1, 0, 2, 0, 3, 1, 3, 3, 2, 1, 0, 1, 1, 2, 1,
        2, 3, 0, 2, 1, 2, 3, 0, 0, 0, 2, 3, 3, 2, 0, 3, 0, 0, 3, 3, 1, 2,
        2, 1, 0, 0, 1, 3, 3, 0, 0, 3, 3, 0, 3, 3, 2, 2, 2, 2, 3, 2, 2, 0,
        0, 2, 3, 3, 2, 2, 3, 1, 3, 3, 3, 3, 0, 2, 2, 3, 2, 1, 0, 1, 2, 2,
        3, 3, 3, 0, 2, 0, 3, 1, 2, 1, 3, 3, 2, 0, 1, 3, 2, 0, 1, 3, 2, 1,
        3, 1, 3, 1, 0, 1, 3, 2, 1, 3, 0, 0, 3, 3, 2, 0, 0, 3, 2, 0, 1, 2,
        3, 2, 2, 1, 1, 2, 0, 2, 2, 1, 2, 1, 2, 0, 2, 3, 1, 0, 0, 1, 1, 1,
        3, 3, 3, 1, 1, 2, 2, 1, 2, 0, 1, 2, 2, 1, 0, 3, 3, 1, 3, 1, 0, 3,
        1, 0, 1, 3, 0, 2, 3, 0, 1, 1, 3, 2, 0, 0, 0, 1, 1, 2, 2, 1, 1, 3,
        2, 0, 0, 2, 1, 3, 1, 2, 1, 3, 2, 2, 0, 0, 1, 1, 3, 1, 2, 2, 0, 2,
        3, 2, 1, 1, 3, 3, 1, 0, 3, 1, 2, 1, 0, 2, 2, 0, 3, 1, 0, 2, 0, 0,
        2, 2, 0, 3, 1, 2, 3, 2, 0, 0, 3,

In [30]:
#Initialise Dueling_DQN Policy
duleing_dqn_policy = agent_dueling_qnet.policy

print(agent_dueling_qnet.policy)

In [31]:
dataset_duel = replay_buffer_duel.as_dataset(num_parallel_calls=3,
                                   sample_batch_size=1,
                                   num_steps=2).prefetch(3)  # Assuming n_step_update = 1 for this example

dataset_iterator = iter(dataset_duel)

sample_observation = next(dataset_iterator)[0].observation.numpy()
observation_dim = sample_observation.shape
print(f"Observation Dimension: {observation_dim}")

Observation Dimension: (1, 2, 40)


### Collect Data for the Dueling Network Implementation using the agent's policy

In [32]:
collect_data_from_api(session_id, duleing_dqn_policy, replay_buffer_duel, initial_collect_steps)

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.01785714, 0.68421054, 0.31578946, 0.01785714, 0.        ,
        1.        , 0.        , 1.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), inf

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.19642857, 1.        , 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.35714287, 1.        , 0.        , 0.00446429, 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.51785713, 0.8947368 , 0.        , 0.01785714, 0.        ,
        1.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.6785714 , 0.84210527, 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

...

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.07142857, 1.        , 0.        , 0.01785714, 1.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.23214285, 1.        , 0.        , 0.01785714, 0.        ,
        1.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, state=(), inf

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.4107143 , 1.        , 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.5714286 , 1.        , 0.        , 0.01785714, 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, state=(), info=())
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 40), dtype=float32, numpy=
array([[0.73214287, 1.        , 0.        , 0.01785714, 1.        ,
        0.        , 0.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int3

### Training Loop for Dueling Q Net Implementation's Agent

In [33]:
batch_size = 60
trajectory_length = 2
# Function to reshape experience
def reshape_experience_for_dqn(experience):
    # Determine the actual batch size of the experience
    actual_batch_size = tf.shape(experience.step_type)[0]
    
    # Determine the adjusted sample size based on the actual batch size
    adjusted_sample_size = actual_batch_size // trajectory_length
    
    # Reshape the experience to [adjusted_sample_size, trajectory_length] + spec.shape to form trajectories
    reshaped_experience = tf.nest.map_structure(
        lambda x: tf.reshape(x, [adjusted_sample_size, trajectory_length] + list(x.shape[1:])),
        experience
    )
    return reshaped_experience

# Number of iterations to train the agent
num_iterations = 80000

# Interval for logging to the console
log_interval = 1000

# Metrics to track during training
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)

for iteration in range(num_iterations):
    
    # Sample a batch of experience from the replay buffer
    experience, _ = replay_buffer_duel.get_next(sample_batch_size=batch_size)
    experience = reshape_experience_for_dqn(experience)
    
    # Train the agent using the sampled experience
    train_loss_value = agent_dueling_qnet.train(experience).loss
    
    # Update training loss metric
    train_loss(train_loss_value)
    
    # Logging and evaluation
    if iteration % log_interval == 0:
        print(f"Iteration {iteration}")
        print(f"Training loss: {train_loss.result()}")
        train_loss.reset_states()


Iteration 0
Training loss: 5.352532386779785
Iteration 1000
Training loss: 7.327658653259277
Iteration 2000
Training loss: 7.159118175506592
Iteration 3000
Training loss: 7.24760627746582
Iteration 4000
Training loss: 7.099137306213379
Iteration 5000
Training loss: 7.327215671539307
Iteration 6000
Training loss: 7.267300605773926
Iteration 7000
Training loss: 7.240710258483887
Iteration 8000
Training loss: 7.124288082122803
Iteration 9000
Training loss: 7.313147068023682
Iteration 10000
Training loss: 7.099369525909424
Iteration 11000
Training loss: 7.494412899017334
Iteration 12000
Training loss: 7.177600383758545
Iteration 13000
Training loss: 7.298490047454834
Iteration 14000
Training loss: 7.220338344573975
Iteration 15000
Training loss: 7.295031547546387
Iteration 16000
Training loss: 7.157742023468018
Iteration 17000
Training loss: 7.358733654022217
Iteration 18000
Training loss: 7.251044750213623
Iteration 19000
Training loss: 7.333632469177246
Iteration 20000
Training loss: 7.4

In [34]:
chosen_policy = agent_dueling_qnet.policy

In [35]:
average_return = compute_average_return(session_id, chosen_policy, num_episodes=100)
print(f"Average Return over 100 episodes: {average_return}")

Average Return over 100 episodes: -97.91704259244972
